In [53]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv("../raw_data/all_features_with_sectorrev.csv")
df

,Unnamed: 0,company_name,TCUID,MI_ID,intensity_scope1_CY2021,intensity_scope1_CY2020,intensity_scope1_CY2019,intensity_scope1_CY2018,intensity_scope1_CY2017,intensity_scope1_CY2016,intensity_scope1_CY2015,intensity_scope1_CY2014,intensity_scope1_CY2013,intensity_scope1_CY2012,intensity_scope1_CY2011,intensity_scope1_CY2010,disclosureCY0,disclosureCY2021,disclosureCY2020,disclosureCY2019,disclosureCY2018,disclosureCY2017,disclosureCY2016,disclosureCY2015,disclosureCY2014,disclosureCY2013,disclosureCY2012,disclosureCY2011,disclosureCY2010,Sector,Industry Group,Industry,Primary Industry,Country / Region Name,S&P Credit Rating,Full Time Employees\n(actual),Total Enterprise Value (CIQ)\n($M),revenueCY0,revenueCY2021,revenueCY2020,revenueCY2019,revenueCY2018,revenueCY2017,revenueCY2016,revenueCY2015,revenueCY2014,revenueCY2013,revenueCY2012,revenueCY2011,revenueCY2010,climate_strategy_scoreFY2022,climate_strategy_scoreFY2021,climate_strategy_scoreFY2020,climate_strategy_scoreFY2019,climate_strategy_scoreFY2018,climate_strategy_scoreFY2017,climate_strategy_scoreFY2016,climate_strategy_scoreFY2015,climate_strategy_scoreFY2014,climate_strategy_scoreFY2013,nuclear_percentage_revenue,thermal_coal_percentage_revenue,ultra_deep_sea_drilling_percentage_revenue,shale_oil_gas_percentage_revenue,coal_percentage_revenue,arctic_drilling_percentage_revenue,oil_sands_percentage_revenue,intensity_scope2CY0,intensity_scope2CY2021,intensity_scope2CY2020,intensity_scope2CY2019,intensity_scope2CY2018,intensity_scope2CY2017,intensity_scope2CY2016,intensity_scope2CY2015,intensity_scope2CY2014,intensity_scope2CY2013,intensity_scope2CY2012,intensity_scope2CY2011,intensity_scope2CY2010,pe_ratio12/31/2021,pe_ratio12/31/2020,pe_ratio12/31/2019,pe_ratio12/31/2018,pe_ratio12/29/2017,pe_ratio12/30/2016,pe_ratio12/31/2015,pe_ratio12/31/2014,pe_ratio12/31/2013,pe_ratio12/31/2012,pe_ratio12/30/2011,pe_ratio12/31/2010,intensity_scope3CY0,intensity_scope3CY2021,intensity_scope3CY2020,intensity_scope3CY2019,intensity_scope3CY2018,intensity_scope3CY2017,intensity_scope3CY2016,intensity_scope3CY2015,intensity_scope3CY2014,intensity_scope3CY2013,intensity_scope3CY2012,intensity_scope3CY2011,intensity_scope3CY2010,Abrasive product manufacturing,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,Advertising and related services,Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air purification and ventilation equipment manufacturing,Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,Alkalies and chlorine manufacturing,All other basic inorganic chemical manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,All other crop farming,All other food manufacturing,"All other forging, stamping, and sintering",All other miscellaneous electrical equipment and component manufacturing,All other miscellaneous manufacturing,"All other miscellaneous professional, scientific, and technical services",All other miscellaneous wood product manufacturing,All other paper bag and coated and treated paper manufacturing,All other petroleum and coal products manufacturing,All other textile product mills,All other transportation equipment manufacturing,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,Ammunition manufacturing,"Amusement parks, arcades, and gambling industries",Analytical laboratory instrument manufacturing,"Animal (except poultry) slaughtering, rendering, and processing","Animal production, except cattle and poultry and eggs",Apparel accessories and other apparel manufacturing,Apparel knitting mills,"Apparel, Piece Goods, and Notions Wholesalers","Architectural, engineering, and related services","Arms, ordnance, and accessories manufacturing",Artificial and synthetic fibers and filaments manufacturing,Asphalt paving mixture an

In [3]:
c= ['nuclear_percentage_revenue',
 'thermal_coal_percentage_revenue',
 'ultra_deep_sea_drilling_percentage_revenue',
 'shale_oil_gas_percentage_revenue',
 'coal_percentage_revenue',
 'arctic_drilling_percentage_revenue',
 'oil_sands_percentage_revenue']

In [4]:
a = df.columns.tolist()
b = ['Abrasive product manufacturing',
 'Accounting, tax preparation, bookkeeping, and payroll services',
 'Adhesive manufacturing',
 'Advertising and related services',
 'Air and gas compressor manufacturing',
 'Air conditioning, refrigeration, and warm air heating equipment manufacturing',
 'Air purification and ventilation equipment manufacturing',
 'Air transportation',
 'Aircraft engine and engine parts manufacturing',
 'Aircraft manufacturing',
 'Alkalies and chlorine manufacturing',
 'All other basic inorganic chemical manufacturing',
 'All other chemical product and preparation manufacturing',
 'All other converted paper product manufacturing',
 'All other crop farming',
 'All other food manufacturing',
 'All other forging, stamping, and sintering',
 'All other miscellaneous electrical equipment and component manufacturing',
 'All other miscellaneous manufacturing',
 'All other miscellaneous professional, scientific, and technical services',
 'All other miscellaneous wood product manufacturing',
 'All other paper bag and coated and treated paper manufacturing',
 'All other petroleum and coal products manufacturing',
 'All other textile product mills',
 'All other transportation equipment manufacturing',
 'Alumina refining and primary aluminum production',
 'Aluminum product manufacturing from purchased aluminum',
 'Ammunition manufacturing',
 'Amusement parks, arcades, and gambling industries',
 'Analytical laboratory instrument manufacturing',
 'Animal (except poultry) slaughtering, rendering, and processing',
 'Animal production, except cattle and poultry and eggs',
 'Apparel accessories and other apparel manufacturing',
 'Apparel knitting mills',
 'Apparel, Piece Goods, and Notions Wholesalers',
 'Architectural, engineering, and related services',
 'Arms, ordnance, and accessories manufacturing',
 'Artificial and synthetic fibers and filaments manufacturing',
 'Asphalt paving mixture and block manufacturing',
 'Asphalt shingle and coating materials manufacturing',
 'Audio and video equipment manufacturing',
 'Automatic environmental control manufacturing',
 'Automobile manufacturing',
 'Automotive equipment rental and leasing',
 'Automotive repair and maintenance, except car washes',
 'Ball and roller bearing manufacturing',
 'Bare printed circuit board manufacturing',
 'Bauxite Mining',
 'Beet sugar manufacturing',
 'Biological product (except diagnostic) manufacturing',
 'Biomass Power Generation',
 'Bituminous Coal Underground Mining',
 'Bituminous Coal and Lignite Surface Mining',
 'Blind and shade manufacturing',
 'Boat building',
 'Book publishers',
 'Bowling centers',
 'Bread and bakery product manufacturing',
 'Breakfast cereal manufacturing',
 'Breweries',
 'Brick, tile, and other structural clay product manufacturing',
 'Broadcast and wireless communications equipment',
 'Broadwoven fabric mills',
 'Broom, brush, and mop manufacturing',
 'Building Material and Garden Equipment and Supplies Dealers',
 'Business support services',
 'Cable and other subscription programming',
 'Car washes',
 'Carbon and graphite product manufacturing',
 'Carbon black manufacturing',
 'Carpet and rug mills',
 'Cattle ranching and farming',
 'Cement manufacturing',
 'Cheese manufacturing',
 'Child day care services',
 'Chocolate and confectionery manufacturing from cacao beans',
 'Clay and nonclay refractory manufacturing',
 'Clothing and Clothing Accessories Stores',
 'Coal Power Generation',
 'Coated and laminated paper, packaging paper and plastics film manufacturing',
 'Coating, engraving, heat treating and allied activities',
 'Coffee and tea manufacturing',
 'Commercial and industrial machinery and equipment rental and leasing',
 'Commercial and industrial machinery and equipment repair and maintenance',
 'Communication and energy wire and cable manufacturing',
 'Community food, housing, and other relief services, including rehabilitation services',
 'Computer storage device manufacturing',
 'Computer systems design services',
 'Computer terminals and other computer peripheral equipment manufacturing',
 'Concrete pipe, brick, and block manufacturing',
 'Confectionery manufacturing from purchased chocolate',
 'Construction machinery manufacturing',
 'Cookie, cracker, and pasta manufacturing',
 'Copper Mining',
 'Copper rolling, drawing, extruding and alloying',
 'Cotton farming',
 'Couriers and messengers',
 'Crown and closure manufacturing and metal stamping',
 'Crude Petroleum and Natural Gas Extraction',
 'Curtain and linen mills',
 'Custom architectural woodwork and millwork manufacturing',
 'Custom computer programming services',
 'Cut and sew apparel contractors',
 'Cut stone and stone product manufacturing',
 'Cutlery, utensil, pot, and pan manufacturing',
 'Cutting tool and machine tool accessory manufacturing',
 'Dairy cattle and milk production',
 'Data processing, hosting, and related services',
 'Death care services',
 'Dental equipment and supplies manufacturing',
 'Directory, mailing list, and other publishers',
 'Distilleries',
 'Dog and cat food manufacturing',
 'Doll, toy, and game manufacturing',
 'Drilling oil and gas wells',
 'Dry, condensed, and evaporated dairy product manufacturing',
 'Dry-cleaning and laundry services',
 'Electric Bulk Power Transmission and Control',
 'Electric Power Distribution',
 'Electric lamp bulb and part manufacturing',
 'Electrical and Electronic Goods Wholesalers',
 'Electricity and signal testing instruments manufacturing',
 'Electromedical and electrotherapeutic apparatus manufacturing',
 'Electron tube manufacturing',
 'Electronic and precision equipment repair and maintenance',
 'Electronic capacitor, resistor, coil, transformer, and other inductor manufacturing',
 'Electronic computer manufacturing',
 'Electronic connector manufacturing',
 'Electronics and Appliance Stores',
 'Elementary and secondary schools',
 'Employment services',
 'Engineered wood member and truss manufacturing',
 'Environmental and other technical consulting services',
 'Fabric coating mills',
 'Fabricated pipe and pipe fitting manufacturing',
 'Facilities support services',
 'Farm machinery and equipment manufacturing',
 'Fats and oils refining and blending',
 'Ferrous metal foundries',
 'Fertilizer manufacturing',
 'Fiber, yarn, and thread mills',
 'Fishing',
 'Fitness and recreational sports centers',
 'Flat glass manufacturing',
 'Flavoring syrup and concentrate manufacturing',
 'Flour milling and malt manufacturing',
 'Fluid milk and butter manufacturing',
 'Fluid power process machinery',
 'Food services and drinking places',
 'Food, Beverage, Health, and Personal Care Stores',
 'Footwear manufacturing',
 'Forest nurseries, forest products, and timber tracts',
 'Frozen food manufacturing',
 'Fruit and vegetable canning, pickling, and drying',
 'Fruit farming',
 'Funds, trusts, and other financial vehicles',
 'Furniture and Home Furnishings Stores',
 'Gasket, packing, and sealing device manufacturing',
 'Gasoline Stations',
 'General Merchandise Stores',
 'General and consumer goods rental except video tapes and discs',
 'Geothermal Power Generation',
 'Glass container manufacturing',
 'Glass product manufacturing made of purchased glass',
 'Gold Ore Mining',
 'Grain farming',
 'Grantmaking, giving, and social advocacy organizations',
 'Greenhouse, nursery, and floriculture production',
 'Grocery and Related Product Wholesalers',
 'Ground or treated mineral and earth manufacturing',
 'Guided missile and space vehicle manufacturing',
 'Handtool manufacturing',
 'Hardware manufacturing',
 'Heating equipment (except warm air furnaces) manufacturing',
 'Heavy duty truck manufacturing',
 'Home health care services',
 'Hospitals',
 'Hotels and motels, including casino hotels',
 'Household cooking appliance manufacturing',
 'Household laundry equipment manufacturing',
 'Household refrigerator and home freezer manufacturing',
 'Hydroelectric Power Generation',
 'Ice cream and frozen dessert manufacturing',
 'In-vitro diagnostic substance manufacturing',
 'Independent artists, writers, and performers',
 'Individual and family services',
 'Industrial gas manufacturing',
 'Industrial mold manufacturing',
 'Industrial process furnace and oven manufacturing',
 'Industrial process variable instruments manufacturing',
 'Institutional furniture manufacturing',
 'Insurance agencies, brokerages, and related activities',
 'Insurance carriers',
 'Internet publishing and broadcasting',
 'Internet service providers and web search portals',
 'Investigation and security services',
 'Iron and steel mills and ferroalloy manufacturing',
 'Iron ore mining',
 'Irradiation apparatus manufacturing',
 'Jewelry and silverware manufacturing',
 'Junior colleges, colleges, universities, and professional schools',
 'Knit fabric mills',
 'Laminated plastics plate, sheet (except packaging), and shape manufacturing',
 'Landfill Gas Power Generation',
 'Lawn and garden equipment manufacturing',
 'Lead Ore and Zinc Ore Mining',
 'Leather and hide tanning and finishing',
 'Legal services',
 'Lessors of nonfinancial intangible assets',
 'Light truck and utility vehicle manufacturing',
 'Lighting fixture manufacturing',
 'Lime and gypsum product manufacturing',
 'Logging',
 'Lumber and Other Construction Materials Wholesalers',
 'Magnetic and optical recording media manufacturing',
 'Management of companies and enterprises',
 'Management, scientific, and technical consulting services',
 'Manufactured home (mobile home) manufacturing',
 'Material handling equipment manufacturing',
 'Mattress manufacturing',
 'Mechanical power transmission equipment manufacturing',
 'Medical and diagnostic labs and outpatient and other ambulatory care services',
 'Medicinal and botanical manufacturing',
 "Men's and boys' cut and sew apparel manufacturing",
 'Metal and other household furniture manufacturing',
 'Metal can, box, and other metal container (light gauge) manufacturing',
 'Metal cutting and forming machine tool manufacturing',
 'Metal tank (heavy gauge) manufacturing',
 'Military armored vehicle, tank, and tank component manufacturing',
 'Mineral wool manufacturing',
 'Mining and oil and gas field machinery manufacturing',
 'Miscellaneous Durable Goods Wholesalers',
 'Miscellaneous Nondurable Goods Wholesalers',
 'Miscellaneous Store Retailers',
 'Miscellaneous nonmetallic mineral products',
 'Monetary authorities and depository credit intermediation',
 'Motion picture and video industries',
 'Motor Vehicle and Machinery, Equipment, and Supplies Wholesalers',
 'Motor Vehicle and Parts Dealers',
 'Motor and generator manufacturing',
 'Motor home manufacturing',
 'Motor vehicle body manufacturing',
 'Motor vehicle parts manufacturing',
 'Motorcycle, bicycle, and parts manufacturing',
 'Museums, historical sites, zoos, and parks',
 'Musical instrument manufacturing',
 'Narrow fabric mills and schiffli machine embroidery',
 'Natural Gas Liquid Extraction',
 'Natural Gas Power Generation',
 'Natural gas distribution',
 'Newspaper publishers',
 'Nickel Mining',
 'Nonchocolate confectionery manufacturing',
 'Nondepository credit intermediation and related activities',
 'Nonferrous metal (except copper and aluminum) rolling, drawing, extruding and alloying',
 'Nonferrous metal foundries',
 'Nonresidential commercial and health care structures',
 'Nonresidential maintenance and repair',
 'Nonresidential manufacturing structures',
 'Nonstore Retailers',
 'Nonupholstered wood household furniture manufacturing',
 'Nonwoven fabric mills',
 'Nuclear Electric Power Generation',
 'Nursing and residential care facilities',
 'Office administrative services',
 'Office furniture manufacturing',
 'Office supplies (except paper) manufacturing',
 'Offices of physicians, dentists, and other health practitioners',
 'Oilseed farming',
 'Ophthalmic goods manufacturing',
 'Optical instrument and lens manufacturing',
 'Ornamental and architectural metal products manufacturing',
 'Other Electric Power Generation',
 'Other Metal Ore Mining',
 'Other accommodations',
 'Other aircraft parts and auxiliary equipment manufacturing',
 'Other amusement and recreation industries',
 'Other animal food manufacturing',
 'Other basic organic chemical manufacturing',
 'Other commercial and service industry machinery manufacturing',
 'Other communications equipment manufacturing',
 'Other computer related services, including facilities management',
 'Other concrete product manufacturing',
 'Other cut and sew apparel manufacturing',
 'Other educational services',
 'Other electronic component manufacturing',
 'Other engine equipment manufacturing',
 'Other fabricated metal manufacturing',
 'Other general purpose machinery manufacturing',
 'Other industrial machinery manufacturing',
 'Other information services',
 'Other leather and allied product manufacturing',
 'Other major household appliance manufacturing',
 'Other nonmetallic mineral mining and quarrying',
 'Other nonresidential structures',
 'Other personal services',
 'Other plastics product manufacturing',
 'Other pressed and blown glass and glassware manufacturing',
 'Other residential structures',
 'Other rubber product manufacturing',
 'Other support services',
 'Packaging machinery manufacturing',
 'Paint and coating manufacturing',
 'Paper mills',
 'Paperboard Mills',
 'Paperboard container manufacturing',
 'Performing arts companies',
 'Periodical publishers',
 'Personal and household goods repair and maintenance',
 'Personal care services',
 'Pesticide and other agricultural chemical manufacturing',
 'Petrochemical manufacturing',
 'Petroleum Power Generation',
 'Petroleum lubricating oil and grease manufacturing',
 'Petroleum refineries',
 'Petroleum, Chemical, and Allied Products Wholesalers',
 'Pharmaceutical preparation manufacturing',
 'Photographic and photocopying equipment manufacturing',
 'Photographic services',
 'Pipeline transportation',
 'Plastics and rubber industry machinery manufacturing',
 'Plastics bottle manufacturing',
 'Plastics material and resin manufacturing',
 'Plastics packaging materials and unlaminated film and sheet manufacturing',
 'Plastics pipe and pipe fitting manufacturing',
 'Plate work and fabricated structural product manufacturing',
 'Plumbing fixture fitting and trim manufacturing',
 'Polystyrene foam product manufacturing',
 'Postal service',
 'Pottery, ceramics, and plumbing fixture manufacturing',
 'Poultry and egg production',
 'Poultry processing',
 'Power boiler and heat exchanger manufacturing',
 'Power, distribution, and specialty transformer manufacturing',
 'Power-driven handtool manufacturing',
 'Prefabricated wood building manufacturing',
 'Primary battery manufacturing',
 'Primary smelting and refining of copper',
 'Primary smelting and refining of nonferrous metal (except copper and aluminum)',
 'Printed circuit assembly (electronic assembly) manufacturing',
 'Printing',
 'Printing ink manufacturing',
 'Promoters of performing arts and sports and agents for public figures',
 'Propulsion units and parts for space vehicles and guided missiles',
 'Pulp mills',
 'Pump and pumping equipment manufacturing',
 'Radio and television broadcasting',
 'Rail transportation (Diesel)',
 'Rail transportation (Electric)',
 'Railroad rolling stock manufacturing',
 'Ready-mix concrete manufacturing',
 'Real estate',
 'Reconstituted wood product manufacturing',
 'Relay and industrial control manufacturing',
 'Residential maintenance and repair',
 'Residential permanent site single- and multi-family structures',
 'Rolling mill and other metalworking machinery manufacturing',
 'Rubber and plastics hoses and belting manufacturing',
 'Sand, gravel, clay, and ceramic and refractory minerals mining and quarrying',
 'Sanitary paper product manufacturing',
 'Sawmills and wood preservation',
 'Scientific research and development services',
 'Seafood product preparation and packaging',
 'Search, detection, and navigation instruments manufacturing',
 'Seasoning and dressing manufacturing',
 'Secondary smelting and alloying of aluminum',
 'Securities, commodity contracts, investments, and related activities',
 'Semiconductor and related device manufacturing',
 'Semiconductor machinery manufacturing',
 'Services to buildings and dwellings',
 'Ship building and repairing',
 'Showcase, partition, shelving, and locker manufacturing',
 'Sign manufacturing',
 'Small electrical appliance manufacturing',
 'Snack food manufacturing',
 'Soap and cleaning compound manufacturing',
 'Soft drink and ice manufacturing',
 'Software publishers',
 'Software, audio, and video media reproducing',
 'Solar Power Generation',
 'Sound recording industries',
 'Soybean and other oilseed processing',
 'Special tool, die, jig, and fixture manufacturing',
 'Specialized design services',
 'Spectator sports',
 'Speed changer, industrial high-speed drive, and gear manufacturing',
 'Sporting and athletic goods manufacturing',
 'Spring and wire product manufacturing',
 'Stationery product manufacturing',
 'Steel product manufacturing from purchased steel',
 'Stone mining and quarrying',
 'Storage battery manufacturing',
 'Sugar cane mills and refining',
 'Sugarcane and sugar beet farming',
 'Support activities for agriculture and forestry',
 'Support activities for oil and gas operations',
 'Support activities for other mining',
 'Support activities for printing',
 'Support activities for transportation',
 'Surgical and medical instrument manufacturing',
 'Surgical appliance and supplies manufacturing',
 'Switchgear and switchboard apparatus manufacturing',
 'Synthetic dye and pigment manufacturing',
 'Synthetic rubber manufacturing',
 'Tar Sands Extraction',
 'Telecommunications',
 'Telephone apparatus manufacturing',
 'Textile and fabric finishing mills',
 'Textile bag and canvas mills',
 'Tire manufacturing',
 'Tobacco product manufacturing',
 'Toilet preparation manufacturing',
 'Totalizing fluid meters and counting devices manufacturing',
 'Transit and ground passenger transportation',
 'Travel arrangement and reservation services',
 'Travel trailer and camper manufacturing',
 'Tree nut farming',
 'Truck trailer manufacturing',
 'Truck transportation',
 'Turbine and turbine generator set units manufacturing',
 'Turned product and screw, nut, and bolt manufacturing',
 'Unlaminated plastics profile shape manufacturing',
 'Upholstered household furniture manufacturing',
 'Uranium-Radium-Vanadium Ore Mining',
 'Urethane and other foam product (except polystyrene) manufacturing',
 'Valve and fittings other than plumbing',
 'Vegetable and melon farming',
 'Vending, commercial, industrial, and office machinery manufacturing',
 'Veneer and plywood manufacturing',
 'Veterinary services',
 'Warehousing and storage',
 'Waste management and remediation services',
 'Watch, clock, and other measuring and controlling device manufacturing',
 'Water transportation',
 'Water, sewage and other systems',
 'Wave & Tidal Power Generation',
 'Wet corn milling',
 'Wind Power Generation',
 'Wineries',
 'Wiring device manufacturing',
 "Women's and girls' cut and sew apparel manufacturing",
 'Wood container and pallet manufacturing',
 'Wood kitchen cabinet and countertop manufacturing',
 'Wood windows and doors and millwork']

In [5]:
df[b] = df[b].replace(np.nan,0.)

In [6]:
df.drop(columns="Unnamed: 0")

,company_name,TCUID,MI_ID,intensity_scope1_CY2021,intensity_scope1_CY2020,intensity_scope1_CY2019,intensity_scope1_CY2018,intensity_scope1_CY2017,intensity_scope1_CY2016,intensity_scope1_CY2015,intensity_scope1_CY2014,intensity_scope1_CY2013,intensity_scope1_CY2012,intensity_scope1_CY2011,intensity_scope1_CY2010,disclosureCY0,disclosureCY2021,disclosureCY2020,disclosureCY2019,disclosureCY2018,disclosureCY2017,disclosureCY2016,disclosureCY2015,disclosureCY2014,disclosureCY2013,disclosureCY2012,disclosureCY2011,disclosureCY2010,Sector,Industry Group,Industry,Primary Industry,Country / Region Name,S&P Credit Rating,Full Time Employees\n(actual),Total Enterprise Value (CIQ)\n($M),revenueCY0,revenueCY2021,revenueCY2020,revenueCY2019,revenueCY2018,revenueCY2017,revenueCY2016,revenueCY2015,revenueCY2014,revenueCY2013,revenueCY2012,revenueCY2011,revenueCY2010,climate_strategy_scoreFY2022,climate_strategy_scoreFY2021,climate_strategy_scoreFY2020,climate_strategy_scoreFY2019,climate_strategy_scoreFY2018,climate_strategy_scoreFY2017,climate_strategy_scoreFY2016,climate_strategy_scoreFY2015,climate_strategy_scoreFY2014,climate_strategy_scoreFY2013,nuclear_percentage_revenue,thermal_coal_percentage_revenue,ultra_deep_sea_drilling_percentage_revenue,shale_oil_gas_percentage_revenue,coal_percentage_revenue,arctic_drilling_percentage_revenue,oil_sands_percentage_revenue,intensity_scope2CY0,intensity_scope2CY2021,intensity_scope2CY2020,intensity_scope2CY2019,intensity_scope2CY2018,intensity_scope2CY2017,intensity_scope2CY2016,intensity_scope2CY2015,intensity_scope2CY2014,intensity_scope2CY2013,intensity_scope2CY2012,intensity_scope2CY2011,intensity_scope2CY2010,pe_ratio12/31/2021,pe_ratio12/31/2020,pe_ratio12/31/2019,pe_ratio12/31/2018,pe_ratio12/29/2017,pe_ratio12/30/2016,pe_ratio12/31/2015,pe_ratio12/31/2014,pe_ratio12/31/2013,pe_ratio12/31/2012,pe_ratio12/30/2011,pe_ratio12/31/2010,intensity_scope3CY0,intensity_scope3CY2021,intensity_scope3CY2020,intensity_scope3CY2019,intensity_scope3CY2018,intensity_scope3CY2017,intensity_scope3CY2016,intensity_scope3CY2015,intensity_scope3CY2014,intensity_scope3CY2013,intensity_scope3CY2012,intensity_scope3CY2011,intensity_scope3CY2010,Abrasive product manufacturing,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,Advertising and related services,Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air purification and ventilation equipment manufacturing,Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,Alkalies and chlorine manufacturing,All other basic inorganic chemical manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,All other crop farming,All other food manufacturing,"All other forging, stamping, and sintering",All other miscellaneous electrical equipment and component manufacturing,All other miscellaneous manufacturing,"All other miscellaneous professional, scientific, and technical services",All other miscellaneous wood product manufacturing,All other paper bag and coated and treated paper manufacturing,All other petroleum and coal products manufacturing,All other textile product mills,All other transportation equipment manufacturing,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,Ammunition manufacturing,"Amusement parks, arcades, and gambling industries",Analytical laboratory instrument manufacturing,"Animal (except poultry) slaughtering, rendering, and processing","Animal production, except cattle and poultry and eggs",Apparel accessories and other apparel manufacturing,Apparel knitting mills,"Apparel, Piece Goods, and Notions Wholesalers","Architectural, engineering, and related services","Arms, ordnance, and accessories manufacturing",Artificial and synthetic fibers and filaments manufacturing,Asphalt paving mixture and block man

In [7]:
df.rename(columns={"Full Time Employees\n(actual)":"Full Time Employees (actual)", "Total Enterprise Value (CIQ)\n($M)":"Total Enterprise Value (CIQ) ($M)"}, inplace=True)

In [8]:
only2020 = ["intensity_scope1_CY2020", "intensity_scope2CY2020", "intensity_scope3CY2020", "disclosureCY2020", "revenueCY2020", "climate_strategy_scoreFY2020", "pe_ratio12/31/2020"]

In [9]:
all_col = ["company_name"] + ["TCUID"] + ["Sector"] + only2020 + b + c

In [10]:
df_2020 = df[all_col]
df_2020[c] = df_2020[c].fillna(0.)

/tmp/ipykernel_9720/842466286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020[c] = df_2020[c].fillna(0.)


In [11]:
df_2020

,company_name,TCUID,Sector,intensity_scope1_CY2020,intensity_scope2CY2020,intensity_scope3CY2020,disclosureCY2020,revenueCY2020,climate_strategy_scoreFY2020,pe_ratio12/31/2020,Abrasive product manufacturing,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,Advertising and related services,Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air purification and ventilation equipment manufacturing,Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,Alkalies and chlorine manufacturing,All other basic inorganic chemical manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,All other crop farming,All other food manufacturing,"All other forging, stamping, and sintering",All other miscellaneous electrical equipment and component manufacturing,All other miscellaneous manufacturing,"All other miscellaneous professional, scientific, and technical services",All other miscellaneous wood product manufacturing,All other paper bag and coated and treated paper manufacturing,All other petroleum and coal products manufacturing,All other textile product mills,All other transportation equipment manufacturing,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,Ammunition manufacturing,"Amusement parks, arcades, and gambling industries",Analytical laboratory instrument manufacturing,"Animal (except poultry) slaughtering, rendering, and processing","Animal production, except cattle and poultry and eggs",Apparel accessories and other apparel manufacturing,Apparel knitting mills,"Apparel, Piece Goods, and Notions Wholesalers","Architectural, engineering, and related services","Arms, ordnance, and accessories manufacturing",Artificial and synthetic fibers and filaments manufacturing,Asphalt paving mixture and block manufacturing,Asphalt shingle and coating materials manufacturing,Audio and video equipment manufacturing,Automatic environmental control manufacturing,Automobile manufacturing,Automotive equipment rental and leasing,"Automotive repair and maintenance, except car washes",Ball and roller bearing manufacturing,Bare printed circuit board manufacturing,Bauxite Mining,Beet sugar manufacturing,Biological product (except diagnostic) manufacturing,Biomass Power Generation,Bituminous Coal Underground Mining,Bituminous Coal and Lignite Surface Mining,Blind and shade manufacturing,Boat building,Book publishers,Bowling centers,Bread and bakery product manufacturing,Breakfast cereal manufacturing,Breweries,"Brick, tile, and other structural clay product manufacturing",Broadcast and wireless communications equipment,Broadwoven fabric mills,"Broom, brush, and mop manufacturing",Building Material and Garden Equipment and Supplies Dealers,Business support services,Cable and other subscription programming,Car washes,Carbon and graphite product manufacturing,Carbon black manufacturing,Carpet and rug mills,Cattle ranching and farming,Cement manufacturing,Cheese manufacturing,Child day care services,Chocolate and confectionery manufacturing from cacao beans,Clay and nonclay refractory manufacturing,Clothing and Clothing Accessories Stores,Coal Power Generation,"Coated and laminated paper, packaging paper and plastics film manufacturing","Coating, engraving, heat treating and allied activities",Coffee and tea manufacturing,Commercial and industrial machinery and equipment rental and leasing,Commercial and industrial machinery and equipment repair and maintenance,Communication and energy wire and cable manufacturing,"Community food, housing, and other relief services, including rehabilitation services",Computer storage device manufacturing,Computer systems design services,Computer terminals and other computer peripheral equipment manufacturing,"Concrete pipe, brick, and block manufacturing",Confectionery manufac

In [12]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16647 entries, 0 to 16646
Data columns (total 461 columns):
 #    Column                                                                                  Non-Null Count  Dtype  
---   ------                                                                                  --------------  -----  
 0    company_name                                                                            16647 non-null  object 
 1    TCUID                                                                                   16647 non-null  float64
 2    Sector                                                                                  16647 non-null  object 
 3    intensity_scope1_CY2020                                                                 16647 non-null  float64
 4    intensity_scope2CY2020                                                                  16646 non-null  float64
 5    intensity_scope3CY2020                                    

In [13]:
df_2020.dropna(subset="intensity_scope2CY2020", inplace=True)

/tmp/ipykernel_9720/3627310822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020.dropna(subset="intensity_scope2CY2020", inplace=True)


In [14]:
df_2020.dropna(subset="revenueCY2020", inplace=True)

/tmp/ipykernel_9720/1971941862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020.dropna(subset="revenueCY2020", inplace=True)


In [15]:
df_2020["climate_strategy_scoreFY2020"] = df_2020.climate_strategy_scoreFY2020.fillna(0.)

/tmp/ipykernel_9720/1602560828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["climate_strategy_scoreFY2020"] = df_2020.climate_strategy_scoreFY2020.fillna(0.)


In [16]:
df_2020["pe_ratio12/31/2020"].replace(np.nan,0., inplace=True)
df_2020["pe_ratio12/31/2020"].replace("NM",0., inplace=True)
df_2020["pe_ratio12/31/2020"] = df_2020["pe_ratio12/31/2020"].astype(dtype="float64")

/tmp/ipykernel_9720/101487573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["pe_ratio12/31/2020"].replace(np.nan,0., inplace=True)
/tmp/ipykernel_9720/101487573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["pe_ratio12/31/2020"].replace("NM",0., inplace=True)
/tmp/ipykernel_9720/101487573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["pe_

In [17]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16639 entries, 0 to 16646
Data columns (total 461 columns):
 #    Column                                                                                  Non-Null Count  Dtype  
---   ------                                                                                  --------------  -----  
 0    company_name                                                                            16639 non-null  object 
 1    TCUID                                                                                   16639 non-null  float64
 2    Sector                                                                                  16639 non-null  object 
 3    intensity_scope1_CY2020                                                                 16639 non-null  float64
 4    intensity_scope2CY2020                                                                  16639 non-null  float64
 5    intensity_scope3CY2020                                    

In [18]:
X = df_2020.drop(columns=["company_name", "TCUID", "intensity_scope1_CY2020", "intensity_scope2CY2020", "intensity_scope3CY2020"])
y = df_2020["intensity_scope1_CY2020"]

In [56]:
X

,Sector,disclosureCY2020,revenueCY2020,climate_strategy_scoreFY2020,pe_ratio12/31/2020,Abrasive product manufacturing,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,Advertising and related services,Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air purification and ventilation equipment manufacturing,Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,Alkalies and chlorine manufacturing,All other basic inorganic chemical manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,All other crop farming,All other food manufacturing,"All other forging, stamping, and sintering",All other miscellaneous electrical equipment and component manufacturing,All other miscellaneous manufacturing,"All other miscellaneous professional, scientific, and technical services",All other miscellaneous wood product manufacturing,All other paper bag and coated and treated paper manufacturing,All other petroleum and coal products manufacturing,All other textile product mills,All other transportation equipment manufacturing,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,Ammunition manufacturing,"Amusement parks, arcades, and gambling industries",Analytical laboratory instrument manufacturing,"Animal (except poultry) slaughtering, rendering, and processing","Animal production, except cattle and poultry and eggs",Apparel accessories and other apparel manufacturing,Apparel knitting mills,"Apparel, Piece Goods, and Notions Wholesalers","Architectural, engineering, and related services","Arms, ordnance, and accessories manufacturing",Artificial and synthetic fibers and filaments manufacturing,Asphalt paving mixture and block manufacturing,Asphalt shingle and coating materials manufacturing,Audio and video equipment manufacturing,Automatic environmental control manufacturing,Automobile manufacturing,Automotive equipment rental and leasing,"Automotive repair and maintenance, except car washes",Ball and roller bearing manufacturing,Bare printed circuit board manufacturing,Bauxite Mining,Beet sugar manufacturing,Biological product (except diagnostic) manufacturing,Biomass Power Generation,Bituminous Coal Underground Mining,Bituminous Coal and Lignite Surface Mining,Blind and shade manufacturing,Boat building,Book publishers,Bowling centers,Bread and bakery product manufacturing,Breakfast cereal manufacturing,Breweries,"Brick, tile, and other structural clay product manufacturing",Broadcast and wireless communications equipment,Broadwoven fabric mills,"Broom, brush, and mop manufacturing",Building Material and Garden Equipment and Supplies Dealers,Business support services,Cable and other subscription programming,Car washes,Carbon and graphite product manufacturing,Carbon black manufacturing,Carpet and rug mills,Cattle ranching and farming,Cement manufacturing,Cheese manufacturing,Child day care services,Chocolate and confectionery manufacturing from cacao beans,Clay and nonclay refractory manufacturing,Clothing and Clothing Accessories Stores,Coal Power Generation,"Coated and laminated paper, packaging paper and plastics film manufacturing","Coating, engraving, heat treating and allied activities",Coffee and tea manufacturing,Commercial and industrial machinery and equipment rental and leasing,Commercial and industrial machinery and equipment repair and maintenance,Communication and energy wire and cable manufacturing,"Community food, housing, and other relief services, including rehabilitation services",Computer storage device manufacturing,Computer systems design services,Computer terminals and other computer peripheral equipment manufacturing,"Concrete pipe, brick, and block manufacturing",Confectionery manufacturing from purchased chocolate,Construction machinery manufacturing,"Cookie, cracker, an

In [19]:
y_baseline = y.mean()
y_baseline

272.5267398280063

In [20]:
mean_absolute_error(y,y_baseline)

2022-09-08 15:52:36.691507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-08 15:52:36.691545: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-08 15:52:36.691563: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VIVOLC): /proc/driver/nvidia/version does not exist
2022-09-08 15:52:36.691951: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=float64, numpy=435.1651561330961>

In [57]:
num_col = sorted(X.select_dtypes(include=["int64", "float64"]).columns)
cat_col = sorted(list(set(X.columns) - set(num_col)))
cat_col

['Sector']

In [22]:
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, num_col),
    ('cat_tr', cat_transformer, cat_col)
],
sparse_threshold=0)

In [23]:
preprocessor

ColumnTransformer(sparse_threshold=0,
                  transformers=[('num_tr', StandardScaler(),
                                 ['Abrasive product manufacturing',
                                  'Accounting, tax preparation, bookkeeping, '
                                  'and payroll services',
                                  'Adhesive manufacturing',
                                  'Advertising and related services',
                                  'Air and gas compressor manufacturing',
                                  'Air conditioning, refrigeration, and warm '
                                  'air heating equipment manufacturing',
                                  'Air purificati...
                                  'All other textile product mills',
                                  'All other transportation equipment '
                                  'manufacturing',
                                  'Alumina refining and primary aluminum '
                                  'production',
                                  'Aluminum product manufacturing from '
                                  'purchased aluminum',
                                  'Ammunition manufacturing',
                                  'Amusement parks, arcades, and gambling '
                                  'industries',
                                  'Analytical laboratory instrument '
                                  'manufacturing', ...]),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sector'])])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [25]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


In [68]:
model = XGBRegressor(n_estimators=1000, max_depth=5, objective="reg:squarederror", eval_metric="mae")
cross_val_score(model, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_error")

array([-121.46260095, -114.77793918, -112.8938739 , -191.33986595,
       -134.91765919])

In [69]:
model2 = XGBRegressor(n_estimators=700, max_depth=5, objective="reg:squarederror", eval_metric="mae")
cross_val_score(model2, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_error")

array([-121.68521866, -114.80991918, -113.0636084 , -191.85190591,
       -135.17876134])

In [71]:
model3 = XGBRegressor(n_estimators=900, max_depth=10, objective="reg:squarederror", eval_metric="mae")
cross_val_score(model3, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_error")

array([-123.77128785, -114.61929502, -112.74603951, -192.41011413,
       -125.51739696])